# Downstream

# Imports

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding
)
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training, PeftModel
from transformers import BitsAndBytesConfig
import torch
import wandb

from tokenize_functions import tokenize_pretrain_dataset

# Configs

In [ ]:
attempt = "attempt_5"

data_path = "../data/pubmed_baseline/"
train_data_path = data_path + "pubmed_train.csv"
val_data_path = data_path + "pubmed_val.csv"

max_len = 360

model_output_dir = f"../models/phi_pubmed_pretrained_{attempt}"

model_id = "microsoft/Phi-3.5-mini-instruct"

## Hyperparameters

In [ ]:
lora_r = 24
lora_alpha = 48
lora_target_modules = ["qkv_proj"]
batch_size = 32
quantization = None
lora_dropout = 0.05
epochs = 3
learning_rate = 1e-5
warmup_steps = 600

# Dataset loading

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
val_df = pd.read_csv(val_data_path)
train_df = pd.read_csv(train_data_path)

In [ ]:
val_set = tokenize_pretrain_dataset(tokenizer, val_df, max_len)
train_set = tokenize_pretrain_dataset(tokenizer, train_df.loc[:200_001, :], max_len)

# Setup

In [ ]:
wandb.init(
    project="pubmed-pretrain",
    name=attempt,
    config={
        "model": model_id,
        "lora_r": lora_r,
        "lora_alpha": lora_alpha,
        "batch_size": batch_size,
        "epochs": epochs,
        "quantization": quantization,
        "lora_target_modules": lora_target_modules
    }
)

## Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

# Train

In [ ]:
TrainingArguments?

In [ ]:
training_args = TrainingArguments(
    output_dir=model_output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    eval_strategy="epoch",  # ✅ eval at each epoch
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    fp16=True,
    report_to="wandb",
    run_name=attempt,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


In [ ]:
trainer.train()

trainer.save_model(model_output_dir + "/final")
tokenizer.save_pretrained(model_output_dir + "/final")

### Merge final model

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

finetuned_model = PeftModel.from_pretrained(base_model, model_output_dir + "/final")
tokenizer = AutoTokenizer.from_pretrained(model_output_dir + "/final", trust_remote_code=True)
merged_model = finetuned_model.merge_and_unload()

In [ ]:
merged_model.save_pretrained(model_output_dir +  "/final_merged", safe_serialization=True)
tokenizer.save_pretrained(model_output_dir + "/final_merged")